In [1]:
%cd gspan_mining
from config import parser
from main import main

/data/experiments/DocRED/gSpan/gspan_mining


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# This is just really handy if you make changes to the files.
%load_ext autoreload
%autoreload 2

In [4]:
args_str = '-s 3 -e 2 -v True -d True -p True -n 10 -w True /data/experiments/DocRED/DataViewer/results/docred_train.data'
FLAGS, _ = parser.parse_known_args(args=args_str.split())

In [ ]:
gs = main(FLAGS)

## plot graphs in database

In [ ]:
for g in gs.graphs.values():
    g.plot()

In [6]:
from gspan import gSpan
min_support = 30
alt = False

In [7]:
gs = gSpan(
    database_file_name='/data/experiments/DocRED/DataViewer/results/docred_train.data',
    min_support=min_support,
    min_num_vertices=2,
    # max_num_vertices=3,
    min_num_edges=1,
#     max_ngraphs=5,
    is_undirected=False,
    verbose=False,
    visualize=False,
    where=True,
    alternative_support=alt
)

"""Run the gSpan algorithm."""
# gathered = gs.run(gather=True)
# print(len(gathered))
# print(list(pdfs.edge for pdfs in gathered[0]))
gs.run()
df1 = gs._report_df
df1.to_pickle(f'all_patterns_support_{min_support}{"_alt" if alt else ""}.pkl')



In [57]:
gathered = gs.run(gather=True)
print(list(sorted(set(pdfs.gid for pdfs in gathered[0]))))

[0, 12, 22, 28, 29, 33, 38, 44, 67, 76, 123, 128, 159, 170, 184, 188, 196, 223, 231, 236, 251, 279, 297, 314, 322, 333, 362, 375, 397, 423, 428, 433, 446, 483, 488, 519, 537, 542, 550, 552, 579, 601, 614, 634, 636, 660, 758, 783, 789, 805, 828, 854, 856, 877, 919, 966, 999, 1015, 1064, 1089, 1112, 1114, 1144, 1156, 1178, 1198, 1205, 1207, 1269, 1295, 1317, 1336, 1355, 1409, 1442, 1448, 1460, 1505, 1509, 1535, 1566, 1567, 1577, 1597, 1648, 1654, 1661, 1677, 1687, 1692, 1698, 1737, 1750, 1752, 1776, 1846, 1864, 1899, 1920, 1925, 1944, 1960, 1998, 2005, 2035, 2046, 2049, 2051, 2052, 2101, 2105, 2117, 2144, 2175, 2188, 2193, 2200, 2210, 2229, 2241, 2270, 2307, 2312, 2361, 2371, 2372, 2374, 2385, 2399, 2441, 2448, 2464, 2471, 2512, 2516, 2554, 2608, 2613, 2618, 2631, 2634, 2681, 2683, 2693, 2701, 2745, 2753, 2787, 2825, 2834, 2840, 2841, 2899, 2917, 2927, 2946, 2964, 2965, 3004, 3049]


In [8]:
from perspective import PerspectiveWidget, Table
import pandas as pd
df1 = pd.read_pickle(f'all_patterns_support_{min_support}{"_alt" if alt else ""}.pkl')
PerspectiveWidget(df1)

CRITICAL:root:Unknown type 'list' for key ''


PerspectiveWidget(columns=['index', 'support', 'description', 'num_vert', 'gids'])

In [9]:
print(df1['description'][58])

t # 57
v 0 LOC 
v 1 LOC 
v 2 LOC 
v 3 LOC 
e 0 1 P131
e 1 2 P131
e 1 3 P17


In [10]:
import itertools

with open('/data/experiments/DocRED/DataViewer/results/docred_train.data', 'r', encoding='utf8') as graph:
    transactions = []
    for line in graph:
        line = line.strip()
        t, *rest = line.split()
        if t == 't':
            c = int(rest[-1])
#             if c >= 5:
#                 break
            transactions.append({'v':[], 'e':[]})
        elif t == 'v':
            transactions[c][t].append(rest[-1])
        else: # t == 'e':
            transactions[c][t].append([int(rest[0]), int(rest[1]), rest[2]])
    


# for t in transactions:
#     print()
#     print(t)
#     for i, e in enumerate(t['e']):
#         print(f'#{i}: {t["v"][e[0]]}_{e[0]} {t["v"][e[1]]}_{e[1]} {e[2]}')

#     print()


def get_edge_patterns(pattern):
    pattern_dict = {'v':[], 'e':[]}
    for line in pattern.split('\n'):
        line = line.strip()
        t, *rest = line.split()
        if t == 't':
            continue
        elif t == 'v':
            pattern_dict[t].append(rest[-1])
        else: # t == 'e':
            pattern_dict[t].append([int(rest[0]), int(rest[1]), rest[2]])
    return [(pattern_dict['v'][start], pattern_dict['v'][end], label) for start, end, label in pattern_dict['e']], pattern_dict


def get_candidates(graph, edge_pattern):
    vs = graph['v']
    es = graph['e']
    candidate_edges = []
    for i, (start, end, label) in enumerate(es):
        if (vs[start], vs[end], label) == edge_pattern:
            candidate_edges.append(i)
    return candidate_edges


def trace(graph2, pattern2, steps, ignore_permutations=False):
#     vs2 = pattern2['v']
    es2 = pattern2
    candidate_paths = itertools.product(*steps)
    candidate_paths = [p for p in candidate_paths if len(p) == len(set(p))]
    gvs = graph2['v']
    ges = graph2['e']
    out = []
#     print("Candidates:", candidate_paths)
    varsets = list()
    for cp2 in candidate_paths:
        # Check if it's valid. We trust the labels and the types.
        # This is basically a type of variable resolution, heh.
        
        variables = dict()
        okay = True
#         print(cp2)
#         print(es2)
        if len(cp2) == len(es2):
            for step, e in zip(cp2, es2):
#                 print(step, e)
                v0, v1, _ = e
                start, end, _ = ges[step]
                if v0 in variables:
#                     print(f"{variables[v0]}!={start} ? (start): {variables}")
                    if variables[v0] != start:
                        okay = False
                        break
                if v1 in variables:
#                     print(f"{variables[v1]}!={end} ? (end): {variables}")
                    if variables[v1] != end:
                        okay = False
                        break

                variables[v0] = start
                variables[v1] = end
            if not okay:
                continue
            varset = set(variables.values())
            # If we didn't assign the same value to different variables
            if len(varset) == len(variables):
                # If we haven't used exactly this set of values before.
                # That is, if we change the order of values and it's still the same pattern,
                # then it's just one instance of that pattern. Permutation invariance.
                if varset in varsets and ignore_permutations:
                    continue
                varsets.append(varset)
                out.append(cp2)
            
    return out


def debug_one(df, index, verbose=True):
    tot = 0
    pattern = df['description'][index]
    support = df['support'][index]
    edge_patterns, pattern_dict = get_edge_patterns(pattern)
    if verbose:
        print(pattern)
    for tr in transactions:
        possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
#         possible_steps = [[10], [3], [5], [9]]
        founds = trace(tr, pattern_dict['e'], possible_steps)
        if verbose:
            print(founds)
        tot += len(founds)
    if verbose:
        print(index, tot, support)


def debug_all(df):
    for i, (pattern, support) in enumerate(zip(df['description'], df['support'])):
        tot = 0
        edge_patterns, pattern_dict = get_edge_patterns(pattern)
        for tr in transactions:
            possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
            founds = trace(tr, pattern_dict['e'], possible_steps)
    #         print(founds)
            tot += len(founds)
        if tot != support:
            print(i, tot, support)
    
# debug_one(df1, 19)
# debug_all(df1)

# from multiprocessing.dummy import Pool, Lock
# def make_corrections(df_in):
#     counts = [0 for _ in range(len(df_in['support']))]
#     lock = Lock()
#     def count_(inp):
# #         print(inp)
#         i, pattern = inp
#         tot = 0
#         edge_patterns, pattern_dict = get_edge_patterns(pattern)
#         for tr in transactions:
#             possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
#             founds = trace(tr, pattern_dict['e'], possible_steps)
#     #         print(founds)
#             tot += len(founds)
#         lock.acquire()
#         print(i, tot)
#         counts[i] = tot
#         lock.release()
#     pool = Pool(23)
# #     print(list(zip(*enumerate(df_in['description']))))
#     pool.map(count_, enumerate(df_in['description']))
#     df_in["counts"] = counts
#     return df_in


In [11]:
# df2 = pd.read_pickle(f'all_patterns_support_{min_support}_wcounts.pkl')

In [12]:
import multiprocessing as mp
import numpy as np

def count_(inp):
#         print(inp)
    counts = np.zeros(len(df1['support']), dtype=int)
    j, tr = inp
    tot = 0
    for i, pattern in enumerate(df1['description']):
        edge_patterns, pattern_dict = get_edge_patterns(pattern)
        possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
        founds = trace(tr, pattern_dict['e'], possible_steps, ignore_permutations=True)
        counts[i] += len(founds)
    print(f'{j},', end='')
    return counts

def make_corrections_2(df_in):
    lock = Lock()
    p = mp.Pool(mp.cpu_count())
    res = p.map_async(count_, enumerate(transactions))
    p.close()
    p.join()
    print()
#     print(np.sum(res.get(), axis=0))
    df_in["counts"] = np.sum(res.get(), axis=0)
    return df_in
    
# df2 = pd.read_pickle(f'all_patterns_support_{min_support}_wcounts_2.pkl')


df2 = make_corrections_2(df1)
df2.to_pickle(f'all_patterns_support_{min_support}_wcounts_noperms_gids.pkl')
PerspectiveWidget(df2)








384,0,128,96,448,224,416,320,704,608,480,288,256,640,672,512,129,449,544,417,576,609,673,321,1,225,641,289,32,513,352,450,674,130,610,226,577,451,545,675,418,353,290,642,385,578,2,514,611,354,546,452,322,291,419,355,579,453,612,292,547,33,386,356,323,515,420,676,454,293,548,613,643,34,357,227,580,324,294,677,421,387,644,228,325,705,549,295,358,645,35,296,455,229,678,422,550,359,297,516,581,551,36,230,423,456,298,582,614,679,646,388,583,299,37,192,389,457,231,647,300,517,584,615,648,193,680,232,424,585,390,616,160,518,425,681,233,617,38,649,682,618,586,619,161,683,519,391,587,234,620,392,194,235,588,131,520,236,650,393,684,39,237,64,621,651,40,65,521,238,195,685,589,41,132,66,394,622,196,239,197,67,623,395,686,522,68,590,396,687,69,624,42,397,688,591,240,398,523,689,241,524,690,625,257,399,525,626,258,526,691,242,400,527,401,692,259,97,198,736,3,402,199,528,403,737,738,4,98,404,200,693,405,99,5,652,653,406,694,100,407,654,101,260,529,102,695,261,103,243,696,244,104,697,105,530,245,655,5

CRITICAL:root:Unknown type 'list' for key ''


PerspectiveWidget(columns=['index', 'support', 'description', 'num_vert', 'gids', 'counts'])

In [50]:
# Now we want to calculate the coverage the patterns have of the dev/eval set.
# Then after we want to do the same for predictions (which will need to be formatted...)

# Document coverage:
# Basically do the counting but mark triples which are included.

def trace_coverage(graph2, pattern2, steps, coverage, ignore_permutations=True):
#     vs2 = pattern2['v']
    es2 = pattern2
    candidate_paths = itertools.product(*steps)
    candidate_paths = [p for p in candidate_paths if len(p) == len(set(p))]
    gvs = graph2['v']
    ges = graph2['e']
    if coverage is None:
        coverage = np.zeros(len(ges), dtype=int)
#     print("Candidates:", candidate_paths)
    varsets = list()
    for cp2 in candidate_paths:
        # Check if it's valid. We trust the labels and the types.
        # This is basically a type of variable resolution, heh.
        
        variables = dict()
        okay = True
#         print(cp2)
#         print(es2)
        if len(cp2) == len(es2):
            for step, e in zip(cp2, es2):
#                 print(step, e)
                v0, v1, _ = e
                start, end, _ = ges[step]
                if v0 in variables:
#                     print(f"{variables[v0]}!={start} ? (start): {variables}")
                    if variables[v0] != start:
                        okay = False
                        break
                if v1 in variables:
#                     print(f"{variables[v1]}!={end} ? (end): {variables}")
                    if variables[v1] != end:
                        okay = False
                        break

                variables[v0] = start
                variables[v1] = end
            if not okay:
                continue
            varset = set(variables.values())
            # If we didn't assign the same value to different variables
            if len(varset) == len(variables):
                # If we haven't used exactly this set of values before.
                # That is, if we change the order of values and it's still the same pattern,
                # then it's just one instance of that pattern. Permutation invariance.
                if varset in varsets and ignore_permutations:
                    continue
                varsets.append(varset)
                coverage[list(cp2)] = 1
    return coverage


def map_coverage(inp):
#     print(inp)
#     counts = np.zeros(len(df2['support']), dtype=int)
    j, tr = inp
    tot = 0
    coverage = None
    for i, pattern in enumerate(df2['description']):
        edge_patterns, pattern_dict = get_edge_patterns(pattern)
        possible_steps = [get_candidates(tr, ep) for ep in edge_patterns]
        coverage = trace_coverage(tr, pattern_dict['e'], possible_steps, coverage)
        if sum(coverage) == len(coverage):
            break
#         counts[i] += len(founds)
#     print(f'{j},')
    return np.sum(coverage), len(coverage)
#     return counts


def find_coverage(trs):
    lock = Lock()
    p = mp.Pool(mp.cpu_count())
    res = p.map_async(map_coverage, enumerate(trs))
    p.close()
    p.join()
    covered, total = zip(*res.get())
    print(f'{sum(covered)}/{sum(total)}: {sum(covered)/sum(total)}%')
#     print(sum(covered), sum(total))
#     print(np.sum(res.get(), axis=0))
#     df_in["counts"] = np.sum(res.get(), axis=0)
#     return df_in

find_coverage(transactions)
# map_coverage((0, transactions[0]))

35318/38180: 0.9250392875851231%


In [49]:
with open('/data/experiments/DocRED/DataViewer/results/docred_dev.data', 'r', encoding='utf8') as graph:
    transactions_dev = []
    for line in graph:
        line = line.strip()
        t, *rest = line.split()
        if t == 't':
            c = int(rest[-1])
#             if c >= 5:
#                 break
            transactions_dev.append({'v':[], 'e':[]})
        elif t == 'v':
            transactions_dev[c][t].append(rest[-1])
        else: # t == 'e':
            transactions_dev[c][t].append([int(rest[0]), int(rest[1]), rest[2]])
find_coverage(transactions_dev)

11358/12323: 0.921691146636371%


In [ ]:
# When I come back from vacation.
# I need to make sure to redo a lot of this.